In [ ]:
!pip install selenium

In [ ]:
import pandas as pd
import os
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import urllib
import pathlib
import datetime

In [ ]:
CHROMEDRIVER_PATH = './chromedriver'
WINDOW_SIZE = "1920,1080"

In [ ]:
# Comman function for init driver
def init_driver(url):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    pathlib.Path('download').mkdir(parents=True, exist_ok=True) 
    
    preferences = {
        "download.default_directory": os.getcwd() + os.path.sep + '/download',
    }

    chrome_options.add_experimental_option('prefs', preferences)

    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,options=chrome_options)
    driver.maximize_window()
    driver.get(url)
    return driver

In [ ]:
def wait_for_page_load(xpath_url):
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath_url))
        WebDriverWait(driver, 5).until(element_present)
    except TimeoutException:
        pass
    except NoSuchElementException:
        print("Element not found")

In [ ]:
#1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search 
    #for guitars

In [ ]:
driver=webdriver.Chrome(r"C:\Users\LENOVO\Desktop\Data\chromedriver.exe")

In [ ]:
product_name = input('guitar')

In [ ]:
driver = init_driver('https://www.amazon.in/')

In [ ]:
# add data to keyword field 
driver.find_element_by_xpath('//input[@id="twotabsearchtextbox"]').send_keys(product_name)

In [ ]:
# hit search 
driver.find_element_by_xpath('//input[@id="nav-search-submit-button"]').click()

In [ ]:
#2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search 
    #results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.


In [ ]:
page_count = 0 
all_urls = []   
while page_count < 3:
    sleep(5)
    # wait for page load
    wait_for_page_load('//div[@class="a-section a-spacing-medium"]')    

    # finding all boxes 
    all_box = driver.find_elements_by_xpath('//div[@class="a-section a-spacing-medium"]')

    
    # Ittrating over box to find details
    for one_box in all_box:

        try:
            url = one_box.find_element_by_xpath('.//h2/a').get_attribute('href')
            all_urls.append(url)
        except:
            pass

        
    # Go to next page
    driver.find_element_by_xpath('//body').send_keys(Keys.CONTROL + Keys.PAGE_DOWN)
    try:
        driver.find_element_by_xpath('//li[@class="a-last"]').click()
    except:
        driver.find_element_by_xpath('//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]').click()
    page_count += 1

In [ ]:
final_data = []
for one_url in all_urls:
    driver.get(one_url)
    
    try:
        sleep(5)
        wait_for_page_load('//span[@id="priceblock_ourprice"]')    


        try:
            brand_name = driver.find_element_by_xpath('//div[@id="productOverview_feature_div"]//span[@class="a-size-base"]').tex
        except:
            brand_name = driver.find_element_by_xpath('//a[@id="bylineInfo"]').text

        product_name = driver.find_element_by_xpath('//h1[@id="title"]').text
        try:
            rating = driver.find_element_by_xpath('//span[@id="acrPopover"]//span//span').get_attribute('innerHTML')
            ratings_count = driver.find_element_by_xpath('//span[@id="acrCustomerReviewText"]').text
        except:
            rating = '-'
            ratings_count = '-'

        try:
            price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]').text
        except:
            try:
                price = driver.find_element_by_xpath('//span[@id="priceblock_dealprice"]').text
            except:
                price = '-'


        return_exchange = driver.find_element_by_xpath('//div[@id="RETURNS_POLICY"]').text
        try:
            expected_delivery = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b').text
        except:
            expected_delivery = '-'
        availability = driver.find_element_by_xpath('//div[@id="availability_feature_div"]').text
        other_details = driver.find_element_by_xpath('//div[@id="feature-bullets"]').text
        product_url = one_url

        final_data.append({
            'brand_name': brand_name,
            'product_name': product_name,
            'rating': rating,
            'ratings_count': ratings_count,
            'price': price,
            'return_exchange': return_exchange,
            'expected_delivery': expected_delivery,
            'availability': availability,
            'other_details': other_details,
            'product_url': product_url,
        })
    except:
        print('Got error in: ',one_url)
        
data = pd.DataFrame(final_data)
data

In [ ]:
#3. Write a python program to access the search bar and search button on images.google.com and scrape 10images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [ ]:
keywords = ['fruits', 'cars', 'Machine Learning']

In [ ]:
def download_image(all_urls):
    for url in all_urls:
        driver.get(url)
        driver.execute_script(f'a = document.createElement("a");a.setAttribute("download","");a.href = "{url}";document.body.appendChild(a);a.click();')

In [ ]:
for key in keywords:
    driver.get('https://images.google.com/')
    driver.find_element_by_xpath('//form//input').send_keys(key)
    driver.find_element_by_xpath('//form//input').send_keys(Keys.ENTER)
    
    
    all_src_list = []
    
    while True:
        all_image_box = driver.find_elements_by_xpath('//div[@id="islmp"]//div[@jsname="DeysSe"]/img')
        
        for one_box in all_image_box:
            try:
                one_box.click()
                sleep(1)
                src = driver.find_element_by_xpath('//div[@id="islsp"]//a/img[contains(@src,"https://")]').get_attribute('src')
                all_src_list.append(src)
            except:
                pass

        driver.find_element_by_xpath('//body').send_keys(Keys.CONTROL + Keys.PAGE_DOWN)
        if len(all_src_list) > 100:
            break
        
    download_image(all_src_list)

In [ ]:
#4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) onwww.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be 
    #scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
    #details is missing then replace it by “- “. Save your results in a dataframe and CSV

In [ ]:
driver.get('https://www.flipkart.com/')

In [ ]:
# close login popup 
try:
    wait_for_page_load('//button[text()="✕"]')
    driver.find_element_by_xpath('//button[text()="✕"]').click()
except:
    pass

In [ ]:
driver.find_element_by_xpath('//input[@name="q"]').send_keys('pixel 4A')
driver.find_element_by_xpath('//button[@type="submit"]').click()

In [ ]:
wait_for_page_load('//div[contains(@data-id,"MOB")]')    

# finding top all boxes 
all_box = driver.find_elements_by_xpath('//div[contains(@data-id,"MOB")]')

In [ ]:
all_urls = [] 
for one_box in all_box:
    all_urls.append(one_box.find_element_by_xpath('.//a').get_attribute('href'))

In [ ]:
final_data = []
for one_url in all_urls:
    driver.get(one_url)
    
    try:
        sleep(5)
        
        driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()

        wait_for_page_load('//h1')    


        brand_name = driver.find_element_by_xpath('//h1').text
        smartphone_name = driver.find_element_by_xpath('//h1').text
        colour = driver.find_element_by_xpath('//td[text()="Color"]/parent::*//li').text
        RAM = driver.find_element_by_xpath('//td[text()="RAM"]/parent::*//li').text
        storage_ROM = driver.find_element_by_xpath('//td[text()="Internal Storage"]/parent::*//li').text
        primary_camera = driver.find_element_by_xpath('//td[text()="Primary Camera"]/parent::*//li').text

        try:    
            secondary_camera = driver.find_element_by_xpath('//td[text()="Secondary Camera"]/parent::*//li').text
        except:
            secondary_camera = '-'

        try:    
            display = driver.find_element_by_xpath('//td[text()="Display Type"]/parent::*//li').text
        except:
            display = '-'

        display_size = driver.find_element_by_xpath('//td[text()="Display Size"]/parent::*//li').text

        resolution = driver.find_element_by_xpath('//td[text()="Resolution"]/parent::*//li').text

        try:
            processor = driver.find_element_by_xpath('//td[text()="Processor Type"]/parent::*//li').text
        except:
            processor = '-'

        try:
            processor_cores = driver.find_element_by_xpath('//td[text()="Processor Core"]/parent::*//li').text
        except:
            processor = '-'

        battery_capacity = driver.find_element_by_xpath('//td[text()="Battery Capacity"]/parent::*//li').text

        price = driver.find_element_by_xpath('//div[@class="CEmiEU"]').text
        product_url = one_url






        final_data.append({
            'brand_name':brand_name,
            'smartphone_name':smartphone_name,
            'colour':colour,
            'RAM':RAM,
            'storage_ROM':storage_ROM,
            'primary_camera':primary_camera,
            'secondary_camera':secondary_camera,
            'display_size':display_size,
            'display':display,
            'resolution':resolution,
            'processor':processor,
            'processor_cores':processor_cores,
            'battery_capacity':battery_capacity,
            'price':price,
            'product_url':product_url,
        })
        
    except:
        print('Got error in: ',one_url)
        
data = pd.DataFrame(final_data)
data

In [ ]:
#5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on googlemaps.


In [ ]:
driver.get('https://www.google.com/maps/')

In [ ]:
city = input(' banka')

In [ ]:
driver.find_element_by_xpath('//input[@id="searchboxinput"]').send_keys(city)
driver.find_element_by_xpath('//input[@id="searchboxinput"]').send_keys(Keys.CONTROL + Keys.ENTER)

In [ ]:
current_url = driver.current_url

In [ ]:
latitude, longitude, _ = current_url.split('@')[1].split('/')[0].split(',')

In [ ]:
print('latitude: ',latitude)
print('longitude: ',longitude)

In [ ]:
    #6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [ ]:
driver.get('https://trak.in/india-startup-funding-investment-2015/')

In [ ]:
all_page_length = driver.find_elements_by_xpath('//div[@class="dataTables_length"]//select')

In [ ]:
for one_page_length in all_page_length:
    select = Select(one_page_length)
    select.select_by_visible_text('100')

In [ ]:
all_table = pd.read_html(driver.page_source)

In [ ]:
df = pd.concat(all_table)
df

In [ ]:
df['Date (dd/mm/yyyy)'] = pd.to_datetime(df['Date (dd/mm/yyyy)'],errors='coerce')

In [ ]:
df.dtypes

In [ ]:
df

In [ ]:
start_time = datetime.datetime(2020, 7, 1)
end_time = datetime.datetime(2020, 9, 30)

In [ ]:
df = df[(start_time < df['Date (dd/mm/yyyy)']) & (df['Date (dd/mm/yyyy)'] < end_time )].sort_values('Date (dd/mm/yyyy)')
df

In [ ]:
df.reset_index()

In [ ]:
#7. Write a program to scrap all the available details of best gaming laptops from digit.in

In [ ]:
driver.get('https://www.digit.in/top-products/best-gaming-laptops-40.html')

In [ ]:
all_laptop_title = driver.find_elements_by_xpath('//div[@class="TopNumbeHeading sticky-footer"]/a')

In [ ]:
all_name = []
for one_title in all_laptop_title:
    all_name.append(one_title.text)
    
name_series = pd.Series(all_name)
name_series

In [ ]:
all_specs = driver.find_elements_by_xpath('//div[@class="Spcs-details"]')

In [ ]:
all_spec_list = []
for one_spec in all_specs:

    os_name = one_spec.find_element_by_xpath('.//td[text()="OS"]/parent::*/td[3]').text
    display = one_spec.find_element_by_xpath('.//td[text()="Display"]/parent::*/td[3]').text
    processor = one_spec.find_element_by_xpath('.//td[text()="Processor"]/parent::*/td[3]').text
    memory = one_spec.find_element_by_xpath('.//td[text()="Memory"]/parent::*/td[3]').text
    weight = one_spec.find_element_by_xpath('.//td[text()="Weight"]/parent::*/td[3]').text
    dimension = one_spec.find_element_by_xpath('.//td[text()="Dimension"]/parent::*/td[3]').text
    graphics_processor = one_spec.find_element_by_xpath('.//td[text()="Graphics Processor"]/parent::*/td[3]').text
    
    all_spec_list.append({
        'os' : os_name,
        'display' : display,
        'processor' : processor,
        'memory' : memory,
        'weight' : weight,
        'dimension' : dimension,
        'graphics_processor' : graphics_processor,
    })
    
df = pd.DataFrame(all_spec_list)
df

In [ ]:
df['name'] = name_series
df

In [ ]:
#8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [ ]:
driver.get('https://www.forbes.com/billionaires/')

In [ ]:
all_list = []

while True:

    all_boxes = driver.find_elements_by_xpath('//div[@class="table-body"]/div/div[contains(@class,"table-row ")]')
    for one_box in all_boxes:

        rank = one_box.find_element_by_xpath('.//div[@class="rank"]').text
        name = one_box.find_element_by_xpath('.//div[@class="personName"]').text
        net_worth = one_box.find_element_by_xpath('.//div[@class="netWorth"]').text
        age = one_box.find_element_by_xpath('.//div[@class="age"]').text
        citizenship = one_box.find_element_by_xpath('.//div[@class="countryOfCitizenship"]').text
        source = one_box.find_element_by_xpath('.//div[@class="source"]').text
        industry = one_box.find_element_by_xpath('.//div[@class="category"]').text

        all_list.append({
            'rank': rank,
            'name': name,
            'net_worth': net_worth,
            'age': age,
            'citizenship': citizenship,
            'source': source,
            'industry': industry,
        })
        
          
    if len(driver.find_elements_by_xpath('//button[contains(@class,"pagination-btn--next pagination-btn--disabled")]')) == 0:
        # we have more pages go to next
        driver.find_element_by_xpath('//button[contains(@class,"pagination-btn--next")]').click()
    else:
        # we are at last page
        break
        
        
df = pd.DataFrame(all_list)
df

In [ ]:
#9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video

In [ ]:
driver.get('https://www.youtube.com/watch?v=kTJczUoc26U')

In [ ]:
wait_for_page_load('//ytd-comment-thread-renderer')

# Loading all comments
i = 1 
while True:
    i += 500
    driver.execute_script(f"window.scrollTo(0, {i});")
        
    all_box = driver.find_elements_by_xpath('//ytd-comment-thread-renderer')
    
    sleep(1)
    
    if len(all_box) > 500:
        break

In [ ]:
all_list = []
for one_box in all_box:
    comment = one_box.find_element_by_xpath('.//ytd-expander[@id="expander"]').text
    upvote = one_box.find_element_by_xpath('.//span[@id="vote-count-middle"]').text
    time = one_box.find_element_by_xpath('.//div[@id="header-author"]//yt-formatted-string/a').text
    
    all_list.append({
        'comment' : comment,
        'upvote' : upvote,
        'time' : time
    })
    
df = pd.DataFrame(all_list)
df


In [ ]:
#10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, 
    #overall reviews, privates from price, dorms from price, facilities and property description

In [ ]:
driver.get('https://www.hostelworld.com/s?q=London,%20England&country=England&city=London&type=city&id=3')

In [ ]:
final_list = []
page_urls = []


while True:
    sleep(5)
    wait_for_page_load('//div[@class="property-card" and @data-v-62f28bda=""]')
    all_boxs = driver.find_elements_by_xpath('//div[@class="property-card" and @data-v-62f28bda=""]')
    for one_box in all_boxs:
        hostel_name = one_box.find_element_by_xpath('.//h2/a').text
        distance_from_city = one_box.find_element_by_xpath('.//a/span').text
        try:
            ratings = one_box.find_element_by_xpath('.//div[contains(@class,"score")]').text
        except:
            ratings = 'No rating'
            
        total_reviews = one_box.find_element_by_xpath('.//div[contains(@class,"reviews")]').text
        overall_reviews = one_box.find_element_by_xpath('.//div[contains(@class,"keyword")]').text
        privates_from_price = one_box.find_elements_by_xpath('.//div[contains(@class,"prices-col")]/a/div')[0].text
        dorms_from_price  = one_box.find_elements_by_xpath('.//div[contains(@class,"prices-col")]/a/div')[1].text
        facilities = one_box.find_element_by_xpath('.//div[contains(@class,"facilities")]').text.split('\n')
        
        page_url = one_box.find_element_by_xpath('.//h2/a').get_attribute('href')
        page_urls.append(page_url)

        final_list.append({
            'hostel_name' : hostel_name,
            'distance_from_city' : distance_from_city,
            'ratings' : ratings,
            'total_reviews' : total_reviews,
            'overall_reviews' : overall_reviews,
            'privates_from_price' : privates_from_price,
            'dorms_from_price' : dorms_from_price,
            'facilities' : facilities,

        })
        
        
    if len(driver.find_elements_by_xpath('//div[contains(@class,"pagination-next disabled")]')) == 0:
        # we have more pages go to next
        driver.find_element_by_xpath('//div[contains(@class,"pagination-next")]').click()
    else:
        # we are at last page
        break
            

df = pd.DataFrame(final_list)
df

In [ ]:
all_desc = []
for one_page in page_urls:
    driver.get(one_page)
    try:
        description = driver.find_element_by_xpath('//div[@class="description-container"]//div[contains(@class,"content")]').text
    except:
        description = ''
        
    all_desc.append(description)    
df_desc = pd.Series(all_desc)
df_desc

In [ ]:
df['description'] = df_desc
df